In [59]:
import itertools
import json
from pathlib import Path

import pandas as pd

In [60]:
def product_experiment_configs(params_record):
    # Generate all possible combinations of parameters
    keys = params_record.keys()
    values = params_record.values()
    for instance in itertools.product(*values):
        yield dict(zip(keys, instance))

In [61]:
common_params = {
    "params.dataset.path": ["bdsaglam/musique"],
    "params.dataset.split": ["validation"],
    "params.qa.model": ["llama-3-70b-tgi"],
    "params.qa.temperature": [
        0.0
    ],
    "params.qa.user_prompt_template": ["cq.txt"],
    "params.qa.n_shot": [
        0,
        # 1,
        # 2,
        3,
    ],
    "params.qa.n_sc": [
        1,
    ],
    "params.run": [
        1,
    ],
}
params_records = [
    {
        "params.qa.system_prompt": [
            "direct/format-few.txt",
            # "direct/excellent-format-few.txt",
        ],
        "params.qa.few_shot_examples": ["semi/direct.json"],
    },
    {
        "params.qa.system_prompt": [
            "cot/format-thought.txt",
            # "cot/excellent-format-thought.txt",
        ],
        "params.qa.few_shot_examples": ["semi/cot.json"],
    },
    {
        "params.qa.system_prompt": [
            "ccot/format-thought.txt",
            # "ccot/excellent-format-thought.txt",
        ],
        "params.qa.few_shot_examples": ["semi/ccot.json"],
    },
    {
        "params.qa.system_prompt": [
            # "cok/original.txt",
            "cok/adapted.txt",
        ],
        "params.qa.few_shot_examples": ["semi/cok.json"],
    },
    {
        "params.qa.system_prompt": [
            "cte/format-triplets-few.txt",
            # "cte/excellent-format-triplets-few.txt",
        ],
        "params.qa.few_shot_examples": ["semi/cte.json"],
    },
]

exp_configs = [
    exp_config
    for params_record in params_records
    for exp_config in product_experiment_configs({**common_params, **params_record})
]
print(f"{len(exp_configs)} experiment configurations")

10 experiment configurations


In [62]:
target_params = exp_configs[0].keys()
target_params

dict_keys(['params.dataset.path', 'params.dataset.split', 'params.qa.model', 'params.qa.temperature', 'params.qa.user_prompt_template', 'params.qa.n_shot', 'params.qa.n_sc', 'params.run', 'params.qa.system_prompt', 'params.qa.few_shot_examples'])

In [63]:
results_path = Path("exps.jsonl")
if results_path.exists():
    with open(results_path) as f:
        experiments = [json.loads(line) for line in f]
else:
    experiments = []

print(f"{len(experiments)} experiments")
next(iter(experiments), None)

5 experiments


{'id': 'c200509b2b27676d059d63d432d335caad8d102e',
 'name': 'gypsy-feel',
 'params.dataset.path': 'bdsaglam/musique',
 'params.dataset.name': 'answerable',
 'params.dataset.split': 'validation',
 'params.qa.model': 'llama-3-70b-tgi',
 'params.qa.temperature': 0.0,
 'params.qa.system_prompt': 'cok/adapted.txt',
 'params.qa.user_prompt_template': 'cq.txt',
 'params.qa.few_shot_examples': 'semi/cok.json',
 'params.qa.n_shot': 0,
 'params.qa.n_sc': 1,
 'params.run': 1,
 'metrics.exact_match': 0.6123293339,
 'metrics.f1': 0.7254104461,
 'metrics.2hops.exact_match': 0.6517571885,
 'metrics.2hops.f1': 0.7596634206,
 'metrics.3hops.exact_match': 0.6092105263,
 'metrics.3hops.f1': 0.7360100684,
 'metrics.4hops.exact_match': 0.4962962963,
 'metrics.4hops.f1': 0.5996315893,
 'metrics.gen_token_count.all.count': 2417.0,
 'metrics.gen_token_count.all.mean': 97.1969383533,
 'metrics.gen_token_count.all.std': 31.9759083653,
 'metrics.gen_token_count.all.min': 24.0,
 'metrics.gen_token_count.all.25%':

In [64]:
df = pd.DataFrame(experiments)

param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]

df.dropna(subset=param_cols, inplace=True, how="any")
df.drop_duplicates(subset=param_cols, inplace=True)

print(f"{len(df)} experiments after preprocessing")

5 experiments after preprocessing


In [65]:
if len(df):
    existing_configs = df[target_params].to_dict(orient="records")
else:
    existing_configs = []

next(iter(existing_configs), None)

{'params.dataset.path': 'bdsaglam/musique',
 'params.dataset.split': 'validation',
 'params.qa.model': 'llama-3-70b-tgi',
 'params.qa.temperature': 0.0,
 'params.qa.user_prompt_template': 'cq.txt',
 'params.qa.n_shot': 0,
 'params.qa.n_sc': 1,
 'params.run': 1,
 'params.qa.system_prompt': 'cok/adapted.txt',
 'params.qa.few_shot_examples': 'semi/cok.json'}

In [66]:
target_params = list({**common_params, **params_records[0]}.keys())
target_params.remove("params.run")
target_params

['params.dataset.path',
 'params.dataset.split',
 'params.qa.model',
 'params.qa.temperature',
 'params.qa.user_prompt_template',
 'params.qa.n_shot',
 'params.qa.n_sc',
 'params.qa.system_prompt',
 'params.qa.few_shot_examples']

In [67]:
# find the missing configurations
missing_configs = [
    dict(kv)
    for kv in list(
        {tuple(sorted(config.items())) for config in exp_configs}
        - {tuple(sorted(config.items())) for config in existing_configs}
    )
]
print(f"{len(missing_configs)} missing configurations")
next(iter(missing_configs), None)


5 missing configurations


{'params.dataset.path': 'bdsaglam/musique',
 'params.dataset.split': 'validation',
 'params.qa.few_shot_examples': 'semi/direct.json',
 'params.qa.model': 'llama-3-70b-tgi',
 'params.qa.n_sc': 1,
 'params.qa.n_shot': 3,
 'params.qa.system_prompt': 'direct/format-few.txt',
 'params.qa.temperature': 0.0,
 'params.qa.user_prompt_template': 'cq.txt',
 'params.run': 1}

In [68]:
def make_command(exp_config):
    run = exp_config["params.run"]
    lines = ["dvc exp run --queue", f"-S run={run}"]
    for target_param in target_params:
        arg_name = target_param.split(".", 1)[-1]
        arg_value = exp_config[target_param]
        lines.append(f"-S {arg_name}='{arg_value}'")

    command = " \\\n    ".join(lines)
    return command

In [69]:
with open("run.sh", "w") as f:
    f.write("#!/bin/sh\n\n")
    for exp_config in missing_configs:
        f.write(make_command(exp_config))
        f.write("\n\n")